In [ ]:
!pip install --upgrade pip -q
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] -q

In [2]:
# Launching ElasticSearch Daemon for DocumentStores

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)

! sleep 30

In [3]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore()

In [4]:
!cp /content/drive/MyDrive/turkish_news_70000.csv /content/

In [7]:
import pandas as pd

file_path = '/content/turkish_news_70000.csv'

tr_data = pd.read_csv(file_path)
data = tr_data.head(20)

def clean_text(text):

  text = text.replace('\\n','')
  text = text.replace('"','')
  text = text.replace('\n','')

  return text

data['text'] = data['text'].apply(clean_text)
docs = []

for value in list(data['text'].values):

  doc = {
      "content":value
  }

  docs.append(doc)

document_store.write_documents(docs)

<ipython-input-7-3a3bd61ce822>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(clean_text)


In [8]:
document_store.describe_documents()

{'count': 20,
 'chars_mean': 1921.75,
 'chars_max': 6860,
 'chars_min': 584,
 'chars_median': 1272.0}

In [ ]:
from haystack.nodes import FARMReader, QuestionGenerator
from haystack.pipelines import QuestionAnswerGenerationPipeline

reader = FARMReader('deepset/roberta-base-squad2')
question_generator = QuestionGenerator()
qa_generator = QuestionAnswerGenerationPipeline(question_generator, reader)

In [10]:
from haystack.nodes import TransformersTranslator
from haystack.pipelines import TranslationWrapperPipeline
from haystack.utils import print_questions

in_translator = TransformersTranslator(model_name_or_path = 'Helsinki-NLP/opus-mt-tr-en')
out_translator = TransformersTranslator(model_name_or_path = 'Helsinki-NLP/opus-mt-tc-big-en-tr')

translation_pipeline = TranslationWrapperPipeline(input_translator = in_translator, output_translator = out_translator, pipeline = qa_generator)

for idx,document in enumerate(document_store):

  print(f'\nContext : {document.content[:100]}\n')
  print('-' * 100)

  questions = translation_pipeline.run(documents = [document])

  print_questions(questions)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/840k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

...

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Translating:   0%|          | 0/2 [00:00<?, ?it/s]

Translating:   0%|          | 0/2 [00:00<?, ?it/s]

Translating:   0%|          | 0/2 [00:00<?, ?it/s]


Generated pairs:
 - Q: Hangi ülke Esed ailesi ile Moskova'ya çekilir?
      A: Rusya
 - Q: Rusya nereye çekiliyor?
      A: Moskova


Sonuçlar Pek Iyi gelmedi. Daha efektik bir method bulunmalı.